# Cell Below is CSV Combiner 

Run the cell below using the triangle button on the top left or clicking in the cell and pressing Shift + Enter to run it. The code will use the files names from the "data" folder. It will assume protein names from each file names by using everything before the first underscore (_) then will combine into one csv file with columns as different proteins from each file. 

Note that there will be a preview of the first 10 rows of what the final csv looks like so you can check it before moving to save it in the following cell. 

In [3]:
import pandas as pd
from IPython.display import FileLink
import os

def merge_protein_csvs(folder_path):
    # Step 1: Get all CSVs
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Step 2: Identify the DAPI file (case-insensitive search)
    dapi_file = next((f for f in csv_files if 'dapi' in f.lower()), None)
    if not dapi_file:
        raise ValueError("No DAPI file found in folder.")

    # Helper function to clean and standardize column names
    def clean_and_standardize_columns(df):
        df.columns = [col.strip().rstrip(',').upper() for col in df.columns]
        col_map = {}
        for col in df.columns:
            if 'POSITION' in col and 'X' in col:
                col_map[col] = 'POSITION X'
            elif 'POSITION' in col and 'Y' in col:
                col_map[col] = 'POSITION Y'
        return df.rename(columns=col_map)

    # Step 3: Load DAPI DataFrame as base
    dapi_df = pd.read_csv(os.path.join(folder_path, dapi_file), skiprows=3)
    dapi_df = clean_and_standardize_columns(dapi_df)

    # Ensure we don't modify DAPI filename in the loop
    remaining_files = [f for f in csv_files if f != dapi_file]

    # Step 4: Process each protein file
    for file in remaining_files:
        filepath = os.path.join(folder_path, file)
        df = pd.read_csv(filepath, skiprows=3)
        df = clean_and_standardize_columns(df)

        # Drop duplicates of position coordinates
        df = df[['POSITION X', 'POSITION Y']].drop_duplicates()

        # Extract protein name and convert to uppercase
        protein_name = file.split('_')[0].upper()

        # Mark presence of (X,Y) in current protein file
        df[protein_name] = 1

        # Merge into base (DAPI) on POSITION X and Y
        dapi_df = dapi_df.merge(df, on=['POSITION X', 'POSITION Y'], how='left')

    # Step 5: Replace NaN with 0 in presence columns
    presence_cols = [col for col in dapi_df.columns
                 if col not in ['POSITION X', 'POSITION Y'] and pd.api.types.is_numeric_dtype(dapi_df[col])]

# Fill NaNs and cast to int
    dapi_df[presence_cols] = dapi_df[presence_cols].fillna(0).astype(int)

    return dapi_df


folder_path = 'data'
combined_csv = merge_protein_csvs(folder_path)
combined_csv


,POSITION X,POSITION Y,POSITION Z,UNIT_X,CATEGORY_X,COLLECTION,TIME_X,SET 1_X,ID,UNNAMED: 9,OVERLAPPED VOLUME TO SURFACES,UNIT_Y,CATEGORY_Y,SURFACES,TIME_Y,SET 1_Y,CD45,CD68,MDSC
0,51257.254,42903.758,26,µm,Surface,Position,1,Class A,3,0,0,µm^3,Surface,cold cluster,1,Class A,0,0,0
1,51619.285,42545.438,12,µm,Surface,Position,1,Class A,4,0,0,µm^3,Surface,cold cluster,1,Class A,1,1,0
2,51489.516,42677.035,14,µm,Surface,Position,1,Class A,5,0,0,µm^3,Surface,cold cluster,1,Class A,0,0,0
3,51618.191,42549.020,16,µm,Surface,Position,1,Class A,9,0,0,µm^3,Surface,cold cluster,1,Class A,1,0,1
4,51434.113,42733.695,17,µm,Surface,Position,1,Class A,10,0,0,µm^3,Surface,cold cluster,1,Class A,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115110,48725.254,40357.273,24,µm,Surface,Position,1,Class B,115202,0,0,µm^3,Surface,cold cluster,1,Class B,0,0,0
115111,48741.340,40339.059,24,µm,Surface,Position,1,Class B,115203,0,0,µm^3,Surface,cold cluster,1,Class B,0,0,0
115112,48718.805,40368.750,25,µm,Surface,Position,1,Class B,115206,0,0,µm^3,Surface,cold cluster,1,Class B,0,0,0
115113,48733.887,40358.613,24,µm,Surface,Position,1,Class B,115207,0,0,µm^3,Surface,cold cluster,1,Class B,0,0,0


# Use cell below to save the combined csv file after ensuring the preview looks correct above. 

Run the cell below by clicking the triangle in the top left or clicking in the cell and pressing shift + enter. Make sure to renmae the file name to whatever you need in the part to left of the green arrow make sure to leave the name in the quotes. 

The combined csv file will be saved in a folder called "combined_csvs" there will also be a hyperlink below that will open up the csv file to make sure you can see where it is stored and view the full file. 

In [32]:
output_dir = "combined_csvs"
os.makedirs(output_dir, exist_ok=True)  

file_name = "final_output.csv" # <----------- Change the file name here (leave the .csv extension)
output_path = os.path.join(output_dir, file_name)

combined_csv.to_csv(output_path, index=False)

FileLink(output_path)


c:\Users\Maltez Lab\Desktop\lab_tools_patrick\maltez_lab_tools\combined_csvs\final_output.csv